# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [84]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [85]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [86]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [87]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [88]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [89]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1b 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [90]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project1b')
except Exception as e:
    print(e)

## Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1 Execution
#### List the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [91]:
## First we delete the table if it does already exists in the database 

query = "DROP TABLE IF EXISTS music_app_history"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [92]:
## Second, we create the table for the first query
## Since the query requests song metadata for a particular session id and item in session
## Session id is chosen as the primary key and item in session is the composite key

query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, \
                  song_length FLOAT, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [93]:
# Third, we load the data into the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # The SQL query to insert the values into the table is executed below
        query = "INSERT INTO music_app_history (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verifying data insertion into the table using SELECT statement

In [94]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
        SELECT artist_name, song_title, song_length 
        FROM music_app_history
        WHERE session_id = 338 AND item_in_session = 4
"""

try:
    query_results = session.execute(query)
    df = pd.DataFrame(query_results._current_rows)
    print(df)    
except Exception as e:
    print(e)

## Uncomment the code below to print normally without getting the results to a dataframe
#for row in query_results:
#    print (row.artist_name, row.song_title, row.song_length)

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### QUERY 2 Execution
#### List the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [95]:
## First we delete the table if it does already exists in the database for ETL process

query = "DROP TABLE IF EXISTS play_log"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [96]:
## Second, we create the table for the first query
## Since the query needs info about the user and the list of songs in order it was played by a particular user in a session
## userid is used as PRIMARY KEY, sessionid is used as COMPOSITE KEY and the itemInSession is used as clustering columns to sort

query = "CREATE TABLE IF NOT EXISTS play_log"
query = query + "(user_id INT, session_id INT, item_in_session INT, \
                  artist_name TEXT, song_title TEXT, first_name TEXT, last_name TEXT, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [97]:
## Third, we load the data into the table
## Variable 'file' is already defined previously

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # The SQL query to insert the values into the table is executed below
        query = "INSERT INTO play_log (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Verifying data insertion into the table using SELECT statement

In [98]:
query = """
        SELECT artist_name, song_title, first_name, last_name
        FROM play_log
        WHERE user_id = 10 AND session_id = 182
"""

try:
    query_results = session.execute(query)
    df = pd.DataFrame(query_results._current_rows)
    print(df)    
except Exception as e:
    print(e)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### QUERY 3 Execution
#### List every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [99]:
## First we delete the table if it does already exists in the database for ETL process

query = "DROP TABLE IF EXISTS song_listeners"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [100]:
## Second, we create the table for the first query
## We use song title as primary key and user id as composite key to ensure only unique values are returned to query

query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + "(song_title TEXT, user_id INT, first_name TEXT,  \
                  last_name TEXT, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [101]:
## Third, we load the data into the table
## Variable 'file' is already defined previously

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # The SQL query to insert the values into the table is executed below
        query = "INSERT INTO song_listeners (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Verifying data insertion into the table using SELECT statement

In [102]:
query = """
        SELECT first_name, last_name
        FROM song_listeners
        WHERE song_title = 'All Hands Against His Own'
"""

try:
    query_results = session.execute(query)
    df = pd.DataFrame(query_results._current_rows)
    print(df)    
except Exception as e:
    print(e)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [103]:
try:
    session.execute("DROP TABLE IF EXISTS music_app_history")
    session.execute("DROP TABLE IF EXISTS play_log")
    session.execute("DROP TABLE IF EXISTS song_listeners")
except Exception as error:
    print(error)  

### Close the session and cluster connection¶

In [104]:
session.shutdown()
cluster.shutdown()